## Test dataset for C2B camera

+ 2019.06.05
+ camera pattern (all 0 -> all light to bucket1)
+ 4 projector patterns consisting of spatial sinusoids
    + `Sinusoids-freq=04_bins=24_subframes=04/`
    + 96 pattern images -> 24/frame
+ 10 static scenes, (1000 frames/scene)
+ 60 ms exposure


In [ ]:
import sys; sys.path.append('..')
import demosaicing as dm
import importlib; importlib.reload(dm)

In [ ]:
import glob
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

In [ ]:
def show_grayscale(img, description=''):
    if len(np.shape(img)) == 2 or np.shape(img)[2] == 1:
        img = np.tile(img[:,:,np.newaxis],(1,1,3))

    img = plt.imshow(img, interpolation='none')
    plt.axis('off')
    plt.title(description)
    

def show_grayscales(images, descriptions=[], layouts=''):

    layouts = "{}1".format(len(images)) if layouts == '' else layouts
    descriptions = ['' for _ in range(len(images))] if descriptions == [] else descriptions

    for i,img in enumerate(images):
        plt.subplot("{}{}".format(layouts, i+1))
        show_grayscale(img, descriptions[i])

    plt.show()

In [ ]:
topdir = "../data/exp60"
n_patterns = 4
n_frames_total = 1000
n_frames_per_pattern = int(n_frames_total/n_patterns)
# n_frames_per_pattern = 20
imgsize = (176, 288)

In [ ]:
scenes = [x for x in os.listdir(topdir)[:2] if not x.startswith('.')]
scenes = {k: f'{os.path.join(topdir, k)}/bucket1*.png' for k in scenes}
scenes = {k: sorted(glob.glob(v)) for k,v in scenes.items()}
assert(all([len(v) == n_frames_total for k,v in scenes.items()]))

avgs = {k: None for k in scenes.keys()}
for k,s in scenes.items():
    patterns = [[s[x] for x in list(np.arange(n_frames_total)) if x%n_patterns == i] \
                for i in range(n_patterns)]
    
    # shape: (4, 176, 288, n_frames_per_pattern)
    video = np.zeros((n_patterns,*imgsize,n_frames_per_pattern), dtype=np.uint8)
    for i,a_pattern in enumerate(patterns):
        for j,img in enumerate(a_pattern[:n_frames_per_pattern]):
            video[i,:,:,j] = cv.imread(img, cv.IMREAD_GRAYSCALE)
            
    video = np.mean(video/255., axis=3)
    video = (np.clip(video,0,1)*255).astype(np.uint8)
    avgs[k] = video

for k,v in avgs.items():
    patterns = [v[i,:,:] for i in range(n_patterns)] + [cv.imread(scenes['candle'][0], cv.IMREAD_GRAYSCALE)]
    desc = [f'object={k} pattern={i}' for i in range(n_patterns)] + ['1 shot']
#     for i in range(n_patterns):
#         show_grayscale(v[i,:,:], f'object={k} pattern={i}')
    show_grayscales(patterns, desc, layouts='32')
#     show_grayscale(cv.imread(scenes['candle'][0], cv.IMREAD_GRAYSCALE), '1 shot')


In [ ]:
img = cv.imread(scenes['candle'][0], cv.IMREAD_GRAYSCALE)
show_grayscale(img)
np.shape(img)

In [ ]:
n_frames/n_patterns == 250